In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col,split

In [2]:
spark = SparkSession.builder \
    .appName("find_google_word") \
    .getOrCreate()

25/10/13 18:00:19 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


In [3]:
df=spark.read.text("data.txt")
df.show(truncate=False)

+------------------------------------------------------------------------------------+
|value                                                                               |
+------------------------------------------------------------------------------------+
|66.249.69.97 --[24/Sep/2014:22:25:44 +0000] "GET http://www.google.com/bot.html”    |
|66.249.69.97 --[24/Sep/2014:22:26:44 +0000] "GET http://www.google.com/how.html”    |
|66.249.69.97 --[24/Sep/2014:22:28:44 +0000] "GET http://dbdmg.polito.it/course.html”|
|71.19.157.179 --[24/Sep/2014:22:30:12 +0000] "GET http://www.google.com/faq.html”   |
|66.249.69.97 --[24/Sep/2014:31:28:44 +0000] "GET http://dbdmg.polito.it/thesis.html”|
+------------------------------------------------------------------------------------+



In [4]:
df=df.withColumn("ip",split(col("value")," ")[0]) \
    .withColumn("date",split("value"," ")[1]) \
    .withColumn("method",split("value"," ")[3]) \
    .withColumn("url",split("value"," ")[4]) \
    .drop("value")
df.show()

+-------------+--------------------+------+--------------------+
|           ip|                date|method|                 url|
+-------------+--------------------+------+--------------------+
| 66.249.69.97|--[24/Sep/2014:22...|  "GET|http://www.google...|
| 66.249.69.97|--[24/Sep/2014:22...|  "GET|http://www.google...|
| 66.249.69.97|--[24/Sep/2014:22...|  "GET|http://dbdmg.poli...|
|71.19.157.179|--[24/Sep/2014:22...|  "GET|http://www.google...|
| 66.249.69.97|--[24/Sep/2014:31...|  "GET|http://dbdmg.poli...|
+-------------+--------------------+------+--------------------+



In [12]:
google_url=df.filter(col("url").contains("google"))
google_url.show(truncate=False)

+-------------+-----------------------+------+-------------------------------+
|ip           |date                   |method|url                            |
+-------------+-----------------------+------+-------------------------------+
|66.249.69.97 |--[24/Sep/2014:22:25:44|"GET  |http://www.google.com/bot.html”|
|66.249.69.97 |--[24/Sep/2014:22:26:44|"GET  |http://www.google.com/how.html”|
|71.19.157.179|--[24/Sep/2014:22:30:12|"GET  |http://www.google.com/faq.html”|
+-------------+-----------------------+------+-------------------------------+



In [14]:
google_url.write.mode("overwrite").csv("./output")